In [1]:
!pip install symspellpy
!pip install fasttext-langdetect
!pip install wordfreq

In [108]:
import numpy as np
import pandas as pd
import random
import ast
from tqdm import tqdm
from IPython import display as ipd

# visualization
import matplotlib.pyplot as plt
from tabulate import tabulate
import seaborn as sns
#system files
import os
import json
import re
import glob

import zipfile
import shutil
import gc
from pydub import AudioSegment
from joblib import Parallel, delayed

from ftlangdetect import detect

from wordfreq import word_frequency ,top_n_list,get_frequency_dict,zipf_frequency
from collections import Counter

from pandarallel import pandarallel
from bnunicodenormalizer import Normalizer 
pandarallel.initialize(progress_bar=True,nb_workers=8)
tqdm.pandas()
bnorm=Normalizer()

import warnings
warnings.filterwarnings('ignore')

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [109]:
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None]) 

In [110]:
def remove_punctuations(my_str):
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’…,<>.‚/?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

In [111]:
word_freq_ress=get_frequency_dict('bn', wordlist='best')

df = pd.DataFrame.from_dict(word_freq_ress,orient='index')
df.reset_index(inplace=True)
df.columns=['word','frequency']


In [112]:
df.frequency.min()

1.0232929922807536e-08

In [113]:
df

,word,frequency
0,না,1.023293e-02
1,করে,1.000000e-02
2,এবং,9.772372e-03
3,00,9.549926e-03
4,ও,8.912509e-03
...,...,...
238738,💗,1.023293e-08
238739,💦,1.023293e-08
238740,😑,1.023293e-08
238741,😬,1.023293e-08


In [114]:
word_freq_ress=get_frequency_dict('bn', wordlist='best')

df = pd.DataFrame.from_dict(word_freq_ress,orient='index')
df.reset_index(inplace=True)
df.columns=['word','frequency']

#df.frequency=df.frequency*65000*100

#df.frequency=df.frequency/df.frequency.min()
# fix word count

df['word'] = df['word'].apply(lambda x : remove_punctuations(x))
df["word"]=df["word"].parallel_apply(lambda x:normalize(x))
df['lang'] = df['word'].apply(detect)
df['lang_bn'] = df['lang'].apply(lambda x:x['lang'])
df=df[df.lang_bn == 'bn']
df = df.drop(['lang','lang_bn'],axis=1)
#df=df[df.word.str.len() >=2]
#df = df.astype({"frequency": int, "word": str})
#df.frequency=df.frequency.replace(0,1)

df_from_ress = df
df_from_ress

,word,frequency
0,না,1.023293e-02
1,করে,1.000000e-02
2,এবং,9.772372e-03
4,ও,8.912509e-03
6,থেকে,7.079458e-03
...,...,...
238724,য়বারের,1.023293e-08
238726,আই,1.023293e-08
238727,বছরে,1.023293e-08
238728,কি,1.023293e-08


In [115]:
df_from_ress.sort_values(by='frequency')

,word,frequency
238730,এ,1.023293e-08
234366,জিপিপি,1.023293e-08
234368,জিমারের,1.023293e-08
234369,জিমেনেস,1.023293e-08
234370,জিম্বাবুয়েসহ,1.023293e-08
...,...,...
6,থেকে,7.079458e-03
4,ও,8.912509e-03
2,এবং,9.772372e-03
1,করে,1.000000e-02


In [116]:
df2 = pd.read_csv("dictionary/bn_full.txt", sep=" ")

df2.reset_index(inplace=True)
df2.columns=['in','word','frequency']

df2 = df2.drop('in',axis=1)
df2['word'] = df2['word'].apply(lambda x : remove_punctuations(x))
df2["word"]=df2["word"].parallel_apply(lambda x:normalize(x))
df2['lang'] = df2['word'].apply(detect)
df2['lang_bn'] = df2['lang'].apply(lambda x:x['lang'])
df2=df2[df2.lang_bn == 'bn']
df2 = df2.drop(['lang','lang_bn'],axis=1)
#df2=df2[df2.word.str.len() >=2]
df_99k = df2
df_99k

,word,frequency
0,আমি,62498
1,না,55643
2,তুমি,34867
3,আমার,33644
4,কি,27254
...,...,...
99986,কারাধ্রাসএর,1
99987,গিরিপথ,1
99988,ফ্রোডোকে,1
99989,কারাধ্রাসের,1


In [117]:
df_99k

,word,frequency
0,আমি,62498
1,না,55643
2,তুমি,34867
3,আমার,33644
4,কি,27254
...,...,...
99986,কারাধ্রাসএর,1
99987,গিরিপথ,1
99988,ফ্রোডোকে,1
99989,কারাধ্রাসের,1


In [118]:
df_from_ress

,word,frequency
0,না,1.023293e-02
1,করে,1.000000e-02
2,এবং,9.772372e-03
4,ও,8.912509e-03
6,থেকে,7.079458e-03
...,...,...
238724,য়বারের,1.023293e-08
238726,আই,1.023293e-08
238727,বছরে,1.023293e-08
238728,কি,1.023293e-08


In [119]:
df_99k['frequency'] = df_99k.frequency/df_99k.frequency.max()

df_99k.frequency = df_99k.frequency*df_from_ress.frequency.max()

In [120]:
df_99k

,word,frequency
0,আমি,1.023293e-02
1,না,9.110546e-03
2,তুমি,5.708848e-03
3,আমার,5.508603e-03
4,কি,4.462355e-03
...,...,...
99986,কারাধ্রাসএর,1.637321e-07
99987,গিরিপথ,1.637321e-07
99988,ফ্রোডোকে,1.637321e-07
99989,কারাধ্রাসের,1.637321e-07


In [121]:
df_ok=df_99k.merge(df_from_ress,on='word',how='outer')

df_ok=df_ok.fillna(df_from_ress.frequency.min())

df_ok['frequency'] = (df_ok['frequency_x'] + df_ok['frequency_y'])/2


df_ok=df_ok.drop(['frequency_x','frequency_y'],axis=1)

df_ok_allright=df_ok.groupby('word')['frequency'].max().reset_index()

df_ok_allright

#df_ok_allright[(df_ok_allright.word.str.len()>=2) & (df_ok_allright.frequency ==0)]['frequency'] = 10.0

#df_ok_allright['frequency'] = df_ok_allright['frequency'] + 100

,word,frequency
0,অ,3.245597e-05
1,অঁতোয়ান,7.102930e-08
2,অঁদ্রে,1.605527e-08
3,অঁযা,1.069375e-06
4,অঁরি,5.167630e-07
...,...,...
224230,য়ের,3.926611e-06
224231,য়েরই,1.141109e-08
224232,য়োদ,1.268427e-08
224233,য়োসা,3.323353e-08


In [122]:
df_ok_allright.sort_values(by='frequency',ascending=False)

,word,frequency
102878,না,9.671738e-03
15635,আমি,7.401906e-03
39870,করে,6.209817e-03
31837,এবং,5.489130e-03
16055,আর,4.930283e-03
...,...,...
59148,গারিঞ্চার,1.023293e-08
20339,ইজারাদাতা,1.023293e-08
83199,ট্রেইস,1.023293e-08
25738,উদ্যানবিদ্যা,1.023293e-08


In [123]:
df_ok_allright.frequency=df_ok_allright.frequency/df_ok_allright.frequency.min()

In [124]:
df_ok_allright.frequency = df_ok_allright.frequency*df_ok_allright.frequency.min()*100

In [125]:
df_ok_allright

,word,frequency
0,অ,317171.814632
1,অঁতোয়ান,694.124776
2,অঁদ্রে,156.898104
3,অঁযা,10450.332811
4,অঁরি,5050.000000
...,...,...
224230,য়ের,38372.304077
224231,য়েরই,111.513439
224232,য়োদ,123.955419
224233,য়োসা,324.770437


In [126]:
df_ok_allright = df_ok_allright.astype({"frequency": int, "word": str})

In [127]:
df_ok_allright

,word,frequency
0,অ,317171
1,অঁতোয়ান,694
2,অঁদ্রে,156
3,অঁযা,10450
4,অঁরি,5050
...,...,...
224230,য়ের,38372
224231,য়েরই,111
224232,য়োদ,123
224233,য়োসা,324


In [101]:
df_ok_allright[df_ok_allright.word.str.len()>=7]

,word,frequency
0,অঁতোয়ান,694
5,অংকগুলো,223
6,অংকটাতে,850
9,অংকনকারী,106
14,অংকুরোদগমের,123
...,...,...
224160,হয়েছেপৌরাণিক,850
224162,হয়েছেসম্প্রতি,850
224164,হয়েথাকে,169
224193,য়ামাদের,850


In [103]:
df2.frequency = df2.frequency/df2.frequency.min()

In [104]:
df2 = df2.astype({"frequency": int, "word": str})

In [105]:
df2

,word,frequency
0,আমি,62498
1,না,55643
2,তুমি,34867
3,আমার,33644
4,কি,27254
...,...,...
99986,কারাধ্রাসএর,1
99987,গিরিপথ,1
99988,ফ্রোডোকে,1
99989,কারাধ্রাসের,1


In [106]:
df2.to_csv('nice_83k.txt',index=False)

df2.sort_values(by='frequency')

,word,frequency
99991,অ্যাডলারকে,1
68549,মাঝিদের,1
68548,সেইপটা,1
68547,সাটটা,1
68546,শিখাতো,1
...,...,...
4,কি,27254
3,আমার,33644
2,তুমি,34867
1,না,55643


In [128]:
#df_ok_allright = df_ok_allright[df_ok_allright.word.str.len()>=2]

# remove weird words
##
df_ok_allright.to_csv('symspell.txt',index=False)

df_ok_allright.sort_values(by='frequency')

,word,frequency
101154,ধের্য,100
176618,রবিবাসরীয়তে,100
183730,লকগেটের,100
211432,সোডাকে,100
45865,কিটগুলি,100
...,...,...
16055,আর,48180556
31837,এবং,53641817
39870,করে,60684639
15635,আমি,72334179
